## Installation

In [ ]:
!pip install git+https://github.com/geraldioeres/detectree2.git
!git clone https://github.com/facebookresearch/detectron2.git detectron2
!pip install --upgrade --force-reinstall rasterio

  Cloning https://github.com/geraldioeres/detectree2.git to /tmp/pip-req-build-bxgkc0d5
  Running command git clone --filter=blob:none --quiet https://github.com/geraldioeres/detectree2.git /tmp/pip-req-build-bxgkc0d5
  Resolved https://github.com/geraldioeres/detectree2.git to commit 21700f4e91a65d102d0dec4795eef728dee20adc
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-t2209wcx/detectron2_f230628f58a447d7a198f38a28865b3f
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-t2209wcx/detectron2_f230628f58a447d7a198f38a28865b3f
  Resolved https://github.com/facebookresearch/detectron2.git to commit b15f64ec4429e23a148972175a0207c5a9ab84cf
  Preparing metadata (setup.py) ... done
  Using cached rasterio-1.3a3-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: rasterio
    Found existing installation: rasterio 1.4.3
    Uninstalling rasterio-1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Flow Code

## Tiling

In [ ]:
from detectree2.preprocessing.tiling import tile_data, to_traintest_folders
from detectree2.models.train import register_train_data, MyTrainer, setup_cfg
import rasterio
import geopandas as gpd

In [ ]:
dataset_dict = {
    # Name : [Folder, Img, Crown]
    "DN" : ["Danum", "Danum", "Danum"],
    "SE" : ["Sepilok", "Sepilok","SepilokEast"],
    "SW" : ["Sepilok", "Sepilok", "SepilokWest"],
    "JE": ["Juaq_East", "JQA_East", "JQA_East"],
    "JW": ["Juaq_West", "JQA_West", "JQA_West"],
    "LN": ["LPB_North", "LPB_North", "LPB_North"]
}

In [ ]:
def data_tiling(dataset_name, dataset_list, buffer=20, tile_width=30, tile_height=30, threshold=0.4):
    """
    Tile data according to the input values

    Args:
        dataset_list (list): list of datasets information
        buffer (int): buffer or overlapping area between tiles
        tile_width (int): width of the tile
        tile_height (int): height of the tile
        threshold (float): threshold for the crown
    """

    appends = str(tile_width) + "_" + str(buffer) + "_" + str(threshold)
    site_path = "/content/drive/MyDrive/Research/data/" + dataset_list[0]

    img_path = site_path + "/rgb/" + dataset_list[1] + ".tif"
    crown_path = site_path + "/crowns/" + dataset_list[2] + ".gpkg"
    out_dir = site_path + '/tiles' + dataset_name + "_" + appends + "/"

    # Read in the tiff file
    data = rasterio.open(img_path)

    # Read in crowns (then filter by an attribute?)
    crowns = gpd.read_file(crown_path)
    crowns = crowns.to_crs(data.crs.data)

    print(data.meta)
    print(crowns.shape)
    print(out_dir)

    tile_data(img_path, out_dir, buffer, tile_width, tile_height, crowns, threshold, mode="rgb")
    data_folder = out_dir
    to_traintest_folders(data_folder, out_dir, test_frac=0.15, strict=False, folds=5)

    return out_dir

In [ ]:
out_dir_list = {}
for ds in dataset_dict:
  out_dir_list[ds] = data_tiling(ds, dataset_dict[ds])

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 84370, 'height': 40665, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'), 'transform': Affine(0.1, 0.0, 600543.5,
       0.0, -0.1, 648673.5)}
(849, 6)
/content/drive/MyDrive/Research/data/Sepilok/tilesSE_30_20_0.4/


  0%|          | 0/37386 [00:00<?, ?it/s]

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': nan, 'width': 84370, 'height': 40665, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'), 'transform': Affine(0.1, 0.0, 600543.5,
       0.0, -0.1, 648673.5)}
(2265, 6)
/content/drive/MyDrive/Research/data/Sepilok/tilesSW_30_20_0.4/


  0%|          | 0/37386 [00:00<?, ?it/s]

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 34992, 'height': 34027, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'), 'transform': Affine(0.013888678056477711, 0.0, 353017.7796537448,
       0.0, -0.013888678056477711, -17666.30547973588)}
(711, 3)
/content/drive/MyDrive/Research/data/Juaq_East/tilesJE_30_20_0.4/


...

  0%|          | 0/132 [00:00<?, ?it/s]

ERROR:detectree2.preprocessing.tiling:Error processing tile JQA_West at (352696, -17715): TopologyException: side location conflict at 352705.88976730197 -17671.028721929568. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile JQA_West at (352696, -17685): TopologyException: side location conflict at 352705.88976730197 -17671.028721929568. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile JQA_West at (352696, -17655): TopologyException: side location conflict at 352705.88976730197 -17671.028721929568. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile JQA_West at (352726, -17715): TopologyException: side location conflict at 352705.88976730197 -17671.028721929568. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile JQA_West at (352726, -17685): TopologyExcept

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 22059, 'height': 44729, 'count': 4, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 50N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",117],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32650"]]'), 'transform': Affine(0.04522973779250793, 0.0, 221246.08743603725,
       0.0, -0.04522973779250793, 84122.96142266122)}
(610, 3)
/content/drive/MyDrive/Research/data/LPB_North/tilesLN_30_20_0.4/


  0%|          | 0/2046 [00:00<?, ?it/s]

ERROR:detectree2.preprocessing.tiling:Error processing tile LPB_North at (221567, 83440): TopologyException: side location conflict at 221595.01873752047 83484.319608770486. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile LPB_North at (221567, 83470): TopologyException: side location conflict at 221595.01873752047 83484.319608770486. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile LPB_North at (221567, 83500): TopologyException: side location conflict at 221595.01873752047 83484.319608770486. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile LPB_North at (221597, 83440): TopologyException: side location conflict at 221595.01873752047 83484.319608770486. This can occur if the input geometry is invalid.
ERROR:detectree2.preprocessing.tiling:Error processing tile LPB_North at (221597, 83470): TopologyException: